In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scikitplot as skplt
import scipy
%matplotlib inline
from scipy.stats import linregress
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from statistics import mean
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
ls

1533148882_failures.csv  HackDay4.ipynb           Untitled.ipynb


In [3]:
df = pd.read_csv("1533148882_failures.csv")

In [4]:
df.head()

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [5]:
df.head(-10)

,date,device,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
0,2015-01-01,S1F01085,0,215630672,56,0,52,6,407438,0,0,7
1,2015-01-01,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,2015-01-01,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,2015-01-01,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,2015-01-01,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
124479,2015-11-02,W1F0G9T7,0,159174456,0,0,0,13,354826,104,104,2
124480,2015-11-02,W1F0JH87,0,130553176,0,4,0,11,354600,0,0,0
124481,2015-11-02,W1F0JXDL,0,80454336,160,0,0,10,357655,0,0,0
124482,2015-11-02,W1F0JY02,0,226850672,0,0,0,8,351379,0,0,0


In [6]:
abs(df.corr()['failure']).sort_values(ascending = False)

failure       1.000000
attribute8    0.119055
attribute7    0.119055
attribute4    0.067398
attribute2    0.052902
attribute5    0.002270
attribute1    0.001984
attribute9    0.001622
attribute3    0.000948
attribute6    0.000550
Name: failure, dtype: float64

In [7]:
df.describe()

,failure,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9
count,124494.000000,1.244940e+05,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000,124494.000000
mean,0.000851,1.223868e+08,159.484762,9.940455,1.741120,14.222693,260172.858025,0.292528,0.292528,12.451524
std,0.029167,7.045960e+07,2179.657730,185.747321,22.908507,15.943021,99151.009852,7.436924,7.436924,191.425623
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000,8.000000,0.000000,0.000000,0.000000
25%,0.000000,6.127675e+07,0.000000,0.000000,0.000000,8.000000,221452.000000,0.000000,0.000000,0.000000
50%,0.000000,1.227957e+08,0.000000,0.000000,0.000000,10.000000,249799.500000,0.000000,0.000000,0.000000
75%,0.000000,1.833084e+08,0.000000,0.000000,0.000000,12.000000,310266.000000,0.000000,0.000000,0.000000
max,1.000000,2.441405e+08,64968.000000,24929.000000,1666.000000,98.000000,689161.000000,832.000000,832.000000,18701.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
date          124494 non-null object
device        124494 non-null object
failure       124494 non-null int64
attribute1    124494 non-null int64
attribute2    124494 non-null int64
attribute3    124494 non-null int64
attribute4    124494 non-null int64
attribute5    124494 non-null int64
attribute6    124494 non-null int64
attribute7    124494 non-null int64
attribute8    124494 non-null int64
attribute9    124494 non-null int64
dtypes: int64(10), object(2)
memory usage: 11.4+ MB


In [9]:
def cutoffgreater(df_in, elem, cutoff):
    
    df_out = df_in[df_in[elem]<cutoff]
    NewTrainingdatasize = df_out.shape[0]
    
    return df_out, NewTrainingdatasize

In [10]:
def plotattributes(dfin):
    
    dfin = dfin.select_dtypes(exclude=['object'])
    
    for elem in dfin:
        plt.hist(dfin[elem], bins=200)
        plt.yscale('log')

        plt.ylabel('counts', fontsize=13)
        plt.xlabel(str(elem), fontsize=13)
        plt.show()

In [11]:
abs(df.corr()['failure'].sort_values(ascending = False))

failure       1.000000
attribute8    0.119055
attribute7    0.119055
attribute4    0.067398
attribute2    0.052902
attribute5    0.002270
attribute1    0.001984
attribute9    0.001622
attribute6    0.000550
attribute3    0.000948
Name: failure, dtype: float64

In [24]:
def GenerateClassPlot(dfin, elem_x, elem_y, model = RandomForestClassifier()):
    
    #model = RandomForestClassifier()
    model.fit(dfin[elem_x].to_frame(), dfin[elem_y])
    x_pred = pd.DataFrame(np.linspace(0, dfin[elem_x].max(), dfin[elem_y].shape[0]))
    y_pred = pd.DataFrame(model.predict(x_pred))
    y_true = dfin[elem_y].to_frame()
    
    accuracy = accuracy_score(y_true, y_pred)
    
    #print(x_pred.shape, y_pred.shape, y_true.shape, df[elem_x].to_frame().shape)
    
    fig1 = plt.figure(figsize = (20,5))
    fig1.suptitle(str(elem_x), fontsize=16)
    a = fig1.add_subplot(141)
    a.plot(x_pred, y_pred, 'r')
    a.scatter(dfin[elem_x].to_frame(), y_true)
    a.set_ylabel('Probability of Failure', fontsize=13)
    a.set_xlabel(str(elem_x), fontsize=13)

    b = fig1.add_subplot(142)
    b.hist(dfin[elem_x], bins=200)
    b.set_yscale('log')
    b.set_ylabel('counts', fontsize=13)
    b.set_xlabel(str(elem_x), fontsize=13)
    
    c = fig1.add_subplot(143)
    fpr, tpr, _ = roc_curve(y_true,  y_pred)
    auc = roc_auc_score(y_true, y_pred)
    c.plot(fpr,tpr,label=str(elem_x)+"\nAUC: "+str(auc)[:6] + "\nAccuracy: "+str(accuracy)[:6])
    c.set_ylabel('True Positive Rate', fontsize=13)
    c.set_xlabel('False Positive Rate', fontsize=13)
    c.legend(loc=4)
    
    
    colors = np.where(df['failure']==0, 'b', 'r')
    d = fig1.add_subplot(144)
    d.scatter(df['date'], df[elem_x], c = colors)
    d.set_ylabel(str(elem_x), fontsize=13)
    d.set_xlabel('Date', fontsize=13)
    plt.show()

In [17]:
df1 = df.select_dtypes(exclude=['object'])
df2 = df.select_dtypes(include=['object'])

In [14]:
df_failure = df[df['failure'] ==1]

In [15]:
df1 = df_failure.select_dtypes(exclude=['object'])
df2 = df_failure.select_dtypes(include=['object'])

In [ ]:
for elem in df1:
    GenerateClassPlot(df1, elem, 'failure')

ValueError: Invalid RGBA argument: array(['b', 'b', 'b', ..., 'b', 'b', 'b'], dtype='<U1')

<Figure size 1440x360 with 4 Axes>

ValueError: Invalid RGBA argument: array(['b', 'b', 'b', ..., 'b', 'b', 'b'], dtype='<U1')

<Figure size 1440x360 with 4 Axes>

In [ ]:
df.shape[0]

In [ ]:
df.duplicated().sum()

In [ ]:
S1F0 = df[df['device'].astype(str).str[:4].str.contains('S1F0')]
S1F0.shape

In [ ]:
S1F1 = df[df['device'].astype(str).str[:4].str.contains('S1F1')]
S1F1.shape

In [ ]:
W1F0 = df[df['device'].astype(str).str[:4].str.contains('W1F0')]
W1F0.shape

In [ ]:
W1F1 = df[df['device'].astype(str).str[:4].str.contains('W1F1')]
W1F1.shape

In [ ]:
Z1F0 = df[df['device'].astype(str).str[:4].str.contains('Z1F0')]
Z1F0.shape

In [ ]:
Z1F1 = df[df['device'].astype(str).str[:4].str.contains('Z1F1')]
Z1F1.shape

In [ ]:
Z1F2 = df[df['device'].astype(str).str[:4].str.contains('Z1F2')]
Z1F2.shape

In [ ]:
GenerateClassPlot(S1F0, 'attribute8', 'failure')

In [ ]:
abs(S1F0.corr()['failure'].sort_values(ascending = False))

In [ ]:
abs(S1F1.corr()['failure'].sort_values(ascending = False))

In [ ]:
abs(W1F0.corr()['failure'].sort_values(ascending = False))

In [ ]:
abs(W1F1.corr()['failure'].sort_values(ascending = False))

In [ ]:
abs(Z1F0.corr()['failure'].sort_values(ascending = False))

In [ ]:
abs(Z1F1.corr()['failure'].sort_values(ascending = False))

In [ ]:
abs(Z1F2.corr()['failure'].sort_values(ascending = False))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
S1F0.hist(bins=100, figsize=(20,15))
plt.show()

In [ ]:
#df.date.unique()

In [ ]:
df.isnull().sum()

In [ ]:
df1 = df.select_dtypes(exclude=['object'])
df2 = df.select_dtypes(include=['object'])

In [ ]:
df1['attribute1'].max()

In [ ]:
plotattributes(df)

In [ ]:
plotattributes(S1F0)

In [ ]:
def cutoffgreater(df_in, elem, cutoff):
    
    df_out = df_in[df_in[elem]<cutoff]
    NewTrainingdatasize = df_out.shape[0]
    
    return df_out, NewTrainingdatasize

In [ ]:
df1, TrainingSize = cutoffgreater(df1, 'attribute2', 60000)
df1['attribute2'] = df1['attribute2'].apply(lambda x:np.log(x+1))

In [ ]:
df1['attribute2'].hist(bins=100, figsize=(5,5))

In [ ]:
df1['attribute3'].unique()

In [ ]:
S1F0 = S1F0.select_dtypes(exclude=['object'])
for elem in S1F0:
    GenerateClassPlot(S1F0, elem, 'failure',GradientBoostingClassifier())

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def OptimalRegression(XData, y):
    
    g = GaussianNB()
    b = BernoulliNB()
    k = KNeighborsClassifier()
    log = LogisticRegression()
    gbc = GradientBoostingClassifier()
    d = DecisionTreeClassifier()
    r = RandomForestClassifier()
    
    algorithms = [g,b,k,log,d,r,gbc]
    names = ['GaussianNB', 'BernoulliNB', 'K Nearest', 'Logistic','Single Tree','Random Forest', 'GradientBoost']

    # fit the data
    for i in range(len(algorithms)):
        algorithms[i] = algorithms[i].fit(XData, y)
    # print metrics
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for i in range(len(algorithms)):
        accuracy.append(accuracy_score(y, algorithms[i].predict(XData)))
        precision.append(precision_score(y, algorithms[i].predict(XData)))
        recall.append(recall_score(y, algorithms[i].predict(XData)))
        f1.append(f1_score(y, algorithms[i].predict(XData)))
    metrics = pd.DataFrame(columns = ['Accuracy', 'Precision', 'Recall', 'F1'], index = names)
    metrics['Accuracy'] = accuracy
    metrics['Precision'] = precision
    metrics['Recall'] = recall
    metrics['F1'] = f1
    return metrics.sort_values('F1', ascending = False)

In [ ]:
OptimalRegression(S1F0.drop(['failure'], axis=1), S1F0['failure'])